In [1]:
import numpy as np
import scipy.interpolate as si
import scipy
import scipy.stats
from scipy import stats
import scipy.signal as ss
import numpy.fft as fft
import pickle
import os
import subprocess
import matplotlib.pyplot as plt

In [2]:
def rotate_portrait(port, phase=0.0, DM=None, P=None, freqs=None,
        nu_ref=np.inf):
    """
    Rotate and/or dedisperse a portrait.
    Positive values of phase and DM rotate the data to earlier phases
        (i.e. it "dedisperses") for freqs < nu_ref.
    When used to dediserpse, rotate_portrait is virtually identical to
        arch.dedisperse() in PSRCHIVE.
    port is a nchan x nbin array of data values.
    phase is a value specifying the amount of achromatic rotation [rot].
    DM is a value specifying the amount of rotation based on the cold-plasma
        dispersion law [cm**-3 pc].
    P is the pulsar period [sec] at the epoch of the data, needed if a DM is
        provided.
    freqs is an array of frequencies [MHz], needed if a DM is provided.
    nu_ref is the reference frequency [MHz] that has zero delay for any value
        of DM.
    """
    #Exact dispersion constant (e**2/(2*pi*m_e*c)) (used by PRESTO).
    Dconst_exact = 4.148808e3  #[MHz**2 cm**3 pc**-1 s]
    
    #"Traditional" dispersion constant (used by PSRCHIVE,TEMPO,PINT).
    Dconst_trad = 0.000241**-1 #[MHz**2 cm**3 pc**-1 s]
    
    #Fitted DM values will depend on this choice.  Choose wisely.
    Dconst = Dconst_trad
    
    pFFT = fft.rfft(port, axis=1)
    for nn in range(len(pFFT)):
        if DM is None and freqs is None:
            pFFT[nn,:] *= np.exp(np.arange(len(pFFT[nn])) * 2.0j * np.pi *
                    phase)
        else:
            D = Dconst * DM / P
            freq = freqs[nn]
            phasor = np.exp(np.arange(len(pFFT[nn])) * 2.0j * np.pi * (phase +
                (D * (freq**-2.0 - nu_ref**-2.0))))
            pFFT[nn,:] *= phasor
    return fft.irfft(pFFT)

# Also need this function to get the frequency values and such
def gen_spline_portrait(mean_prof, freqs, eigvec, tck, nbin=None):
    """
    Generate a model portrait from make_spline_model(...) output.
    mean_prof is the mean profile.
    freqs are the frequencies at which to build the model.
    eigvec are the eigenvectors providing the basis for the B-spline curve.
    tck is a tuple containing knot locations, B-spline coefficients, and spline
        degree (output of si.splprep(...)).
    nbin is the number of phase bins to use in the model; if different from
        len(mean_prof), a resampling function is used.
    """
    if not eigvec.shape[1]:
        port = np.tile(mean_prof, len(freqs)).reshape(len(freqs),
                len(mean_prof))
    else:
        proj_port = np.array(si.splev(freqs, tck, der=0, ext=0)).T
        delta_port = np.dot(proj_port, eigvec.T)
        port = delta_port + mean_prof
    if nbin is not None:
        if len(mean_prof) != nbin:
            shift = 0.5 * (nbin**-1 - len(mean_prof)**-1)
            port = ss.resample(port, nbin, axis=1)
            port = rotate_portrait(port, shift) #ss.resample introduces shift!
    return port

def read_spline_model(modelfile, freqs=None, nbin=None, quiet=False):
    """
    Read-in a model created by make_spline_model(...).
    If only modelfile is specified, returns the contents of the pickled model:
        (model name, source name, datafile name from which the model was
        created, mean profile vector used in the PCA, the eigenvectors, and the
        'tck' tuple containing knot locations, B-spline coefficients, and
        spline degree)
    Otherwise, builds a model based on the input frequencies using the function
        gen_spline_portrait(...).
    modelfile is the name of the make_spline_model(...)-type of model file.
    freqs in an array of frequencies at which to build the model; these should
        be in the same units as the datafile frequencies, and they should be
        within the same bandwidth range (cf. the knot vector).
    nbin is the number of phase bins to use in the model; by default it uses
        the number in modelfile.
    quiet=True suppresses output.
    """
    if freqs is None:
        read_only = True
    else:
        read_only = False
    if not quiet:
        print("Reading model from %s..."%modelfile)
    modelname, source, datafile, mean_prof, eigvec, tck = \
            pickle.load(open(modelfile, 'rb'))
    if read_only:
        return (modelname, source, datafile, mean_prof, eigvec, tck)
    else:
        return (modelname,
                gen_spline_portrait(mean_prof, freqs, eigvec, tck, nbin))

In [3]:
infile = open("/home/jdc0059/SimulatorProj/templatefiles/J1918-0642/J1918-0642.Rcvr1_2.GUPPI.12y.x.avg_port.spl")
new_arr = pickle.load(infile)

In [4]:
model,psr,avg_port,mean_prof,eigvec,tck = read_spline_model("/home/jdc0059/SimulatorProj/templatefiles/J1918-0642/J1918-0642.Rcvr1_2.GUPPI.12y.x.avg_port.spl")
freqs_Lband = np.linspace(1162.1, 1749.6, 45)

Reading model from /home/jdc0059/SimulatorProj/templatefiles/J1918-0642/J1918-0642.Rcvr1_2.GUPPI.12y.x.avg_port.spl...


In [5]:
print(np.shape(mean_prof))
print(np.shape(eigvec))
print(np.shape(tck))

(2048,)
(2048, 1)
(3,)


In [6]:
modelname, prof_models_interp_Lband = read_spline_model("/home/jdc0059/SimulatorProj/templatefiles/J1918-0642/J1918-0642.Rcvr1_2.GUPPI.12y.x.avg_port.spl", freqs=freqs_Lband, nbin=2048)

Reading model from /home/jdc0059/SimulatorProj/templatefiles/J1918-0642/J1918-0642.Rcvr1_2.GUPPI.12y.x.avg_port.spl...


In [7]:
#print(new_arr)
#print(type(new_arr))

#np.save("/home/jdc0059/SimulatorProj/templatefiles/J1918-0642/J1918-0642.Rcvr1_2.GUPPI.12y.x.avg_port.spl", new_arr)

In [8]:
infile = open("/home/jdc0059/SimulatorProj/templatefiles/J1918-0642/J1918-0642.Rcvr_800.GUPPI.12y.x.avg_port.spl")
new_arr = pickle.load(infile)

In [9]:
model,psr,avg_port,mean_prof,eigvec,tck = read_spline_model("/home/jdc0059/SimulatorProj/templatefiles/J1918-0642/J1918-0642.Rcvr1_2.GUPPI.12y.x.avg_port.spl")
freqs_800 = np.linspace(750, 850, 64)

Reading model from /home/jdc0059/SimulatorProj/templatefiles/J1918-0642/J1918-0642.Rcvr1_2.GUPPI.12y.x.avg_port.spl...


In [10]:
modelname, prof_models_interp_800 = read_spline_model("/home/jdc0059/SimulatorProj/templatefiles/J1918-0642/J1918-0642.Rcvr_800.GUPPI.12y.x.avg_port.spl", freqs=freqs_800, nbin=2048)

Reading model from /home/jdc0059/SimulatorProj/templatefiles/J1918-0642/J1918-0642.Rcvr_800.GUPPI.12y.x.avg_port.spl...


In [11]:
print(np.shape(prof_models_interp_Lband), np.shape(prof_models_interp_800))

((45, 2048), (64, 2048))


In [12]:
# save profiles
"""np.save("B1855+09_Lband_Profs", prof_models_interp_Lband)
np.save("B1855+09_Lband_Freqs", freqs_Lband)
np.save("B1855+09_430_Profs", prof_models_interp_430)
np.save("B1855+09_430_Freqs", freqs_430)"""
# save profiles with constricted version
#np.save("B1855+09_Lband_Profs_NANOfreq_64", prof_models_interp_Lband)
#np.save("B1855+09_Lband_Freqs_NANOfreq_64", freqs_Lband)
#np.save("B1855+09_430_Profs_NANOfreq_64", prof_models_interp_430)
#np.save("B1855+09_430_Freqs_NANOfreq_64", freqs_430)


'np.save("B1855+09_Lband_Profs", prof_models_interp_Lband)\nnp.save("B1855+09_Lband_Freqs", freqs_Lband)\nnp.save("B1855+09_430_Profs", prof_models_interp_430)\nnp.save("B1855+09_430_Freqs", freqs_430)'